<a href="https://colab.research.google.com/github/meenakshikagda/Data-visualization/blob/master/ENCODE4_report_colab_facets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import statsmodels.api as sm
import matplotlib.pyplot as plt
from vega_datasets import data
import seaborn as sns
import json
import requests


In [ ]:
#@title Install the facets_overview pip package.
!pip install facets-overview

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#If you need to provide key pairs, uncomment this block:
#AUTHID = "###" 
#AUTHPW = "###" 
#keypair = (AUTHID, AUTHPW)

In [ ]:
def ListToString(x):
    x = list(set(x))
    nodups = ''
    for item in x:
        if item:
            nodups += item + ','
    return nodups[:-1]

data_table = []

In [23]:
#get list of experiment accessions

exp_list = []
data_table = []



#url = 'https://www.encodeproject.org/search/?type=Experiment&format=json' ### this returns all experiments and is huge
url_exclude_all_control = 'https://www.encodeproject.org/search/?type=Experiment&award.rfa=ENCODE4&status!=deleted&status!=replaced&status!=archived&status!=revoked&control_type!=*&field=award.rfa&field=accession&field=assay_title&field=target.label&field=biosample_ontology.term_name&field=lab.title&field=award.project&field=status&field=replicates.biological_replicate_number&field=date_released&field=date_created&field=possible_controls&field=award.rfa&field=award.pi.title&field=lab&field=possible_controls&field=replicates&field=biosample_ontology&field=internal_tags&field=date_created&field=date_submitted&field=date_released&field=status&limit=all&format=json'

url_exclude_all_control_json = requests.get(url_exclude_all_control)
#To get in progress or submitted data, need to use your auth as below instead of the above line:
#url_exclude_all_control_json = requests.get(url_exclude_all_control, auth=keypair) 


url_exclude_all_control_exp_json = url_exclude_all_control_json.json().get('@graph')


url_single_cell_unit = 'https://www.encodeproject.org/search/?type=SingleCellUnit&award.rfa=ENCODE4&status!=deleted&status!=replaced&status!=archived&status!=revoked&control_type!=*&field=award.rfa&field=accession&field=assay_title&field=target.label&field=biosample_ontology.term_name&field=lab.title&field=award.project&field=status&field=replicates.biological_replicate_number&field=date_released&field=date_created&field=possible_controls&field=award.rfa&field=award.pi.title&field=lab&field=possible_controls&field=replicates&field=biosample_ontology&field=internal_tags&field=date_created&field=date_submitted&field=date_released&field=status&limit=all&format=json'
url_single_cell_unit = requests.get(url_single_cell_unit)

#To get in progress or submitted data, need to use your auth as below instead of the above line:
#url_single_cell_unit = requests.get(url_single_cell_unit, auth=keypair)
url_single_cell_unit = url_single_cell_unit.json().get('@graph')


url_control_dnaseq = 'https://www.encodeproject.org/search/?type=Experiment&award.rfa=ENCODE4&status!=deleted&status!=replaced&status!=archived&status!=revoked&assay_title=DNase-seq&assay_title=ATAC-seq&control_type=*&format=json&frame=embedded&limit=all&format=json'
url_control_dnaseq_json = requests.get(url_control_dnaseq)

#To get in progress or submitted data, need to use your auth as below instead of the above line:
#url_control_dnaseq_json = requests.get(url_control_dnaseq, auth=keypair)
url_control_dnaseq_exp_json = url_control_dnaseq_json.json().get('@graph')

exps_json = url_exclude_all_control_exp_json + url_control_dnaseq_exp_json + url_single_cell_unit





In [24]:
for exp_json in exps_json:
    # get project, grant, lab, accession, assay_type
    project = exp_json.get('award').get('rfa')
    
    try:
        grant = (exp_json.get('award').get('pi').get('title')).split()[-1]
    except AttributeError:
        grant = 'n/a'
    lab = ((exp_json.get('lab').get('title')).split(',')[0]).split()[-1]
    accession = exp_json.get('accession')
    assay_type = exp_json.get('assay_title')

    # get controls metadata
    controls_json = exp_json.get('possible_controls')
    controls = ''
    for control in controls_json:
        control_accession = control.get('accession')
        controls += control_accession + ', '
    controls = controls[:-2]
    
    # get target control_type
    if exp_json.get('control_type'):
        control_type = exp_json.get('control_type')
    else:
        control_type = ''

    # get target metadata
    if exp_json.get('target'):
        target = exp_json.get('target').get('label')
    else:
        target = ''

    # get replicate, biosample, donor, organism, life stage, age
    replicates = exp_json.get('replicates')

    biological_replicates_count = []
    donors_list = []
    species_list = []
    life_stage_list = []
    age_list = []

    for replicate in replicates:
        # list of bioreps
        biorep_number = replicate.get('biological_replicate_number')
        biological_replicates_count.append(biorep_number)

        # get biosample
        if replicate.get('library').get('biosample'):
            biosample = replicate.get('library').get('biosample')
            
            # donors_list
            if biosample.get('donor'):
                donor = biosample.get('donor').get('accession')
                donors_list.append(donor)
            
            # species_list
            if biosample.get('organism').get('name'):
                biosample_species = biosample.get('organism').get('name')
                species_list.append(biosample_species)
            
            # life_stage_list
            if biosample.get('life_stage'):
                life_stage = biosample.get('life_stage')
                life_stage_list.append(life_stage)
                
            # age_list
            if biosample.get('age_display'):
                age = biosample.get('age_display')
                age_list.append(age)
            
        else:
            biosample = ''

    # get replicate count
    biological_replicates_count = len(set(biological_replicates_count))
    if biological_replicates_count == 0:
        biological_replicates_count = ''
    
    # get donors
    donors = ListToString(donors_list)

    # get biosample metadata
    biosample_type = exp_json.get('biosample_ontology').get('classification')
    biosample_term_name = exp_json.get('biosample_ontology').get('term_name')


    # get ENTEx status
    internal_tags = ''.join(exp_json.get('internal_tags'))
    if internal_tags == 'ENTEx':
        entex_status = 'ENTEx'
    else:
        entex_status = ''

    # get treatments
    if biosample:
        if biosample.get('treatments'):
            treatments_json = biosample.get('treatments')
            trt_term_names = []
            for trt in treatments_json:
                trt_term_name = trt.get('treatment_term_name')
                trt_term_names.append(trt_term_name)
            treatments = ListToString(trt_term_names)
        else:
            treatments = ''

    # get species, life_stage, age
    species = ListToString(species_list)
    life_stages = ListToString(life_stage_list)
    ages = ListToString(age_list)

    # get status
    status = exp_json.get('status')

    # get date_created, date_submitted, date_released
    date_created = (exp_json.get('date_created')).split('T')[0]

    if exp_json.get('date_submitted'):
        date_submitted = exp_json.get('date_submitted')
    else:
        date_submitted = ''

    if exp_json.get('date_released'):
        date_released = exp_json.get('date_released')
    else:
        date_released = ''

    row = [project, grant, lab, accession, assay_type, controls, target, biological_replicates_count, donors, biosample_type, biosample_term_name, entex_status, treatments, species, life_stages, ages, status, date_created, date_submitted, date_released, control_type]

    data_table.append(row)

In [25]:
reportDF = pd.DataFrame(data_table, columns = ['Project', 'Grant', 'Lab', 'Accession', 'Assay type', 'Controls', 'Target', 'Replicates', 'Donors', 'Biosample type', 'Biosample', 'ENTEx status', 'Treatment', 'Species', 'Life stage', 'Age', 'Status', 'Date created', 'Date submitted', 'Date released', 'Control type'])
reportDF

,Project,Grant,Lab,Accession,Assay type,Controls,Target,Replicates,Donors,Biosample type,...,ENTEx status,Treatment,Species,Life stage,Age,Status,Date created,Date submitted,Date released,Control type
0,ENCODE4,Bernstein,Bernstein,ENCSR068BIF,Mint-ChIP-seq,ENCSR975DSD,H3K36me3,1,ENCDO833KUM,primary cell,...,,"Interleukin-2,anti-CD3 and anti-CD28 coated beads",human,adult,42 years,released,2022-02-10,,2022-04-29,
1,ENCODE4,Graveley,Graveley,ENCSR163RXI,CRISPR RNA-seq,ENCSR347NKX,RBM8A,2,ENCDO000AAD,cell line,...,,,human,adult,53 years,released,2022-02-01,,2022-02-14,
2,ENCODE4,Wold,Wold,ENCSR209TIR,total RNA-seq,,,1,ENCDO741BVY,tissue,...,,,human,adult,90 or above years,released,2022-04-14,,2022-06-03,
3,ENCODE4,Wold,Wold,ENCSR826FNO,total RNA-seq,,,1,ENCDO999WDR,tissue,...,,,human,adult,84 years,released,2022-04-14,,2022-06-03,
4,ENCODE4,Stamatoyannopoulos,Stamatoyannopoulos,ENCSR338GYD,DNase-seq,,,2,ENCDO509HIY,tissue,...,,,mouse,unknown,25 days,released,2022-03-02,,2022-04-07,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7249,ENCODE4,Wold,Wold,ENCSR671PKV,scRNA-seq,,,1,ENCDO956IXV,tissue,...,,,mouse,embryonic,10.5 days,released,2021-08-19,,2021-08-20,
7250,ENCODE4,Wold,Wold,ENCSR399TQS,scRNA-seq,,,1,ENCDO956IXV,tissue,...,,,mouse,embryonic,13.5 days,released,2021-08-19,,2021-08-20,
7251,ENCODE4,Wold,Wold,ENCSR525NKE,scRNA-seq,,,1,ENCDO956IXV,tissue,...,,,mouse,embryonic,13.5 days,released,2021-08-19,,2021-08-20,
7252,ENCODE4,Wold,Wold,ENCSR654FZO,scRNA-seq,,,1,ENCDO956IXV,tissue,...,,,mouse,embryonic,15.5 days,released,2021-08-19,,2021-08-20,


In [26]:
reportDF.describe()

,Replicates
count,7254.000000
mean,1.383788
std,0.556913
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,9.000000


In [ ]:
# reportDF to tsv
#reportDF.to_csv(path_or_buf = 'pandas_report_ENCODE4_July_18_2022.tsv', sep = '\t', index = False)

In [27]:
#Note here we will not get any in progress or submitted numbers because I haven't used my auth keys to parse the json

# Status per Grant

# get list of grant names
grant_names = list(set(reportDF['Grant']))
grant_names.sort()

statusDF = pd.DataFrame()

# get counts for each status per grant
for name in grant_names:
    grantDF = reportDF.loc[reportDF['Grant'] == name]
    status_dict = dict(grantDF['Status'].value_counts())
    status_dict['Totals'] = len(grantDF)
    variableDF = pd.DataFrame(status_dict, columns = ['in progress', 'submitted', 'released', 'Totals'], index=[name])
    statusDF = statusDF.append(variableDF)

statusDF = statusDF.fillna(0)
statusDF.loc['Totals'] = statusDF.sum()
statusDF

,in progress,submitted,released,Totals
Aiden,0,0,50,50
Bernstein,0,0,1951,1951
Graveley,0,0,438,438
Greenleaf,0,0,110,110
Lee,0,0,52,52
Lis,0,0,49,49
Ljungman,0,0,97,97
Myers,0,0,452,452
Pritchard,0,0,100,100
Reddy,0,0,11,11


In [ ]:
# statusDF to tsv
#statusDF.to_csv(path_or_buf = 'status_report_ENCODE4_July_18_2022.tsv', sep = '\t')

In [ ]:
# status by date

#          '####-##-##'
last_day = '2021-11-04'

created_recently = reportDF[(reportDF['Date created'] > last_day)]
submitted_recently = reportDF[(reportDF['Date submitted'] > last_day)]
released_recently = reportDF[(reportDF['Date released'] > last_day)]

#print(len(created_recently), len(submitted_recently), len(released_recently))
print(last_day)
print('Recently created:', len(created_recently))
print('Recently submitted:', len(submitted_recently))
print('Recently released:', len(released_recently))

In [29]:
# Display the visualization for the data.
from IPython.core.display import display, HTML

jsonstr = reportDF.to_json(orient='records')
HTML_TEMPLATE = """
        <script src="https://cdnjs.cloudflare.com/ajax/libs/webcomponentsjs/1.3.3/webcomponents-lite.js"></script>
        <link rel="import" href="https://raw.githubusercontent.com/PAIR-code/facets/1.0.0/facets-dist/facets-jupyter.html">
        <facets-dive id="elem" height="600"></facets-dive>
        <script>
          var data = {jsonstr};
          document.querySelector("#elem").data = data;
        </script>"""
html = HTML_TEMPLATE.format(jsonstr=jsonstr)
display(HTML(html))